<a href="https://colab.research.google.com/github/ashrafwkhalil/EonLabs-Assessment/blob/main/Eon_Labs_Data_Collection_Assessment.ipynb#scrollTo=cba778ac" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytrends

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=5c783135ccd4fe4a2c43fe96be97cf321aba0129bc3626ab9945b95ee9041ca6
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


## Add Necessary Imports

In [2]:
import pandas as pd
from pytrends.request import TrendReq

## Connect to Google

In [3]:
pytrend= TrendReq()

## Build Payload and Make Initial Hourly Request
I use the get_historical_interest API method to get hourly interest data related to the 'Bitcoin' keyword. I will make no other API calls for the rest of this notebook, being that this data is sufficient to derive all other desired data. I had to set the 'sleep' parameter to 0.2 to ensure that some time passes between successive API calls so that I don't get rate limited by Google. Also, to ensure this lengthy API call isn't done multiple times by accident, I check to see if df has already been initialized.

In [6]:
kw_list = ['bitcoin']
try:
  df 
except:
  pytrend.build_payload(kw_list)
  df = pytrend.get_historical_interest(kw_list, year_start = 2015, month_start = 1, day_start = 1, hour_start = 0, year_end = 2022, month_end = 10, day_end = 1, hour_end = 11, sleep = 0.3)

## Hourly Interest
Basically just re-naming and dropping isPartial Column, also resetting index so date can be used for groupby

In [7]:
hourly_interest = df[['bitcoin']]
hourly_interest = hourly_interest.reset_index()
hourly_interest.rename(columns = {'date':'Hour', 'bitcoin':'Interest'}, inplace = True)
hourly_interest

,Hour,Interest
0,2015-01-01 00:00:00,29
1,2015-01-01 01:00:00,42
2,2015-01-01 02:00:00,71
3,2015-01-01 03:00:00,63
4,2015-01-01 04:00:00,36
...,...,...
65398,2022-08-17 20:00:00,95
65399,2022-08-17 21:00:00,82
65400,2022-08-17 22:00:00,78
65401,2022-08-17 23:00:00,66


## Daily Interest
Just grouping hourly data by day using a custom grouper that groups based on a frequency; in this case, day is the frequency. This returns the daily average interest.

In [8]:
daily_interest = hourly_interest.groupby(pd.Grouper(key = 'Hour', freq = 'D')).mean()
daily_interest = daily_interest.reset_index()
daily_interest.rename(columns = {'Hour':'Day'}, inplace = True)
daily_interest

,Day,Interest
0,2015-01-01,45.250000
1,2015-01-02,53.833333
2,2015-01-03,49.750000
3,2015-01-04,61.708333
4,2015-01-05,63.416667
...,...,...
2782,2022-08-14,75.708333
2783,2022-08-15,79.375000
2784,2022-08-16,75.458333
2785,2022-08-17,76.333333


## Weekly Interest
Just grouping hourly data by day using a custom grouper that groups based on a frequency; in this case, day is the frequency. This returns the daily average interest.

In [9]:
weekly_interest = daily_interest.groupby(pd.Grouper(key = 'Day', freq = 'W')).mean()
weekly_interest = weekly_interest.reset_index()
weekly_interest.rename(columns = {'Day':'Week Start (Sunday)'}, inplace = True)
weekly_interest

,Week Start (Sunday),Interest
0,2015-01-04,52.635417
1,2015-01-11,45.270000
2,2015-01-18,48.523333
3,2015-01-25,47.832143
4,2015-02-01,60.405476
...,...,...
394,2022-07-24,69.595000
395,2022-07-31,73.419762
396,2022-08-07,72.957619
397,2022-08-14,79.036190


## Create and Write to CSV
I will create 4 csv files, because I am unsure what the assessment requires. I will make one where all the tables are appended to one another, and one for each table.

In [13]:
hourly_interest.to_csv('interest_data_hourly.csv', header = True, index = False)
hourly_interest.to_csv('interest_data.csv', header = True, index = False)
daily_interest.to_csv('interest_data_daily.csv', header = True, index = False)
daily_interest.to_csv('interest_data.csv', header = True, mode = 'a', index = False)
weekly_interest.to_csv('interest_data_weekly.csv', header = True, index = False)
weekly_interest.to_csv('interest_data.csv', header = True, mode = 'a', index = False)

In [14]:
display = pd.read_csv('interest_data.csv')
display

,Hour,Interest
0,2015-01-01 00:00:00,29
1,2015-01-01 01:00:00,42
2,2015-01-01 02:00:00,71
3,2015-01-01 03:00:00,63
4,2015-01-01 04:00:00,36
...,...,...
68586,2022-07-24,69.595
68587,2022-07-31,73.4197619047619
68588,2022-08-07,72.95761904761905
68589,2022-08-14,79.03619047619047


In [15]:
display = pd.read_csv('interest_data_hourly.csv')
display

,Hour,Interest
0,2015-01-01 00:00:00,29
1,2015-01-01 01:00:00,42
2,2015-01-01 02:00:00,71
3,2015-01-01 03:00:00,63
4,2015-01-01 04:00:00,36
...,...,...
65398,2022-08-17 20:00:00,95
65399,2022-08-17 21:00:00,82
65400,2022-08-17 22:00:00,78
65401,2022-08-17 23:00:00,66


In [16]:
display = pd.read_csv('interest_data_daily.csv')
display

,Day,Interest
0,2015-01-01,45.250000
1,2015-01-02,53.833333
2,2015-01-03,49.750000
3,2015-01-04,61.708333
4,2015-01-05,63.416667
...,...,...
2782,2022-08-14,75.708333
2783,2022-08-15,79.375000
2784,2022-08-16,75.458333
2785,2022-08-17,76.333333


In [17]:
display = pd.read_csv('interest_data_weekly.csv')
display

,Week Start (Sunday),Interest
0,2015-01-04,52.635417
1,2015-01-11,45.270000
2,2015-01-18,48.523333
3,2015-01-25,47.832143
4,2015-02-01,60.405476
...,...,...
394,2022-07-24,69.595000
395,2022-07-31,73.419762
396,2022-08-07,72.957619
397,2022-08-14,79.036190
